In [1]:
# Bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Prever a probabilidade de default
def prever_default(dados, modelo, scaler):
    # Criar um DataFrame a partir do dicionário de entrada
    df = pd.DataFrame([dados])

    # Padronizar as características dos novos dados usando o mesmo scaler
    dados_scaled = scaler.transform(df)

    # Obter a probabilidade de default usando o modelo
    probabilidade_default = modelo.predict_proba(dados_scaled)[:, 1]

    # Determinar a classe predita (0 ou 1) com base em um limiar (por exemplo, 0.5)
    predicao = int(probabilidade_default > 0.5)

    # Retornar a predição em um formato de dicionário
    resultado = {"default": predicao}

    return resultado

# Abrindo arquivo
df = pd.read_csv('dataset/dataset_2021-5-26-10-14.csv', on_bad_lines='skip',sep = '\t' ,encoding = 'utf-8')
# Fromatando arquivo para data frame
data = pd.DataFrame.from_dict(df)

# Ajustando colunas com valores monetários
colunas_valor = [coluna for coluna in data.columns if 'valor' in coluna or 'ioi' in coluna]
data[colunas_valor] = data[colunas_valor].astype('double').round(2)

# Escolhendo dados para análise
dados_analise = [data['default_3months'],data['ioi_3months'],data['valor_por_vencer'],data['valor_vencido'],data['valor_quitado']
                 ,data['quant_protestos'],data['valor_protestos'],data['quant_acao_judicial'],data['acao_judicial_valor']
                 ,data['participacao_falencia_valor'],data['dividas_vencidas_valor'],data['dividas_vencidas_qtd']
                 ,data['falencia_concordata_qtd'],data['valor_total_pedido'],data['default']]

df = pd.DataFrame(dados_analise).T

# Separando variável alvo (default) das características
X = df.drop('default', axis=1)
y = df['default']

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizando as características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criando e treinando modelo de Regressão Logística
modelo = LogisticRegression(max_iter=1000, random_state=42)
modelo.fit(X_train_scaled, y_train)

# Fazendo predições no conjunto de teste
previsoes = modelo.predict(X_test_scaled)
probabilidades_default = modelo.predict_proba(X_test_scaled)[:, 1]

# Avaliando o desempenho do modelo
acuracia = accuracy_score(y_test, previsoes)
relatorio_classificacao = classification_report(y_test, previsoes)
roc_auc = roc_auc_score(y_test, probabilidades_default)

# Limpando última coluna
n_dados = df.drop(df.columns[-1], axis=1)

# Selecionando cliente que deseja consulta chance de default de 0 a 117272
cliente = 5
n_dados = n_dados.iloc[cliente]

# Chamando função de predição
resultado_predicao = prever_default(n_dados, modelo, scaler)
# Imprimindo resultado
print(resultado_predicao)

{'default': 0}


/tmp/ipykernel_1249336/299664427.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicao = int(probabilidade_default > 0.5)
